In [3]:
#!pip install linearmodels
#!pip install econtools
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

(16127, 31)


In [4]:
df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1


for i in range(len(auth_list)):
    name = 'trend_pa_'+str(i+1)
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j, 'id_auth'] == i+1 :
            df.loc[j, name] = df.loc[j,'trend']
        if df.loc[j, 'authority_code']==3090272 or df.loc[j, 'authority_code']==3070001:
            df.loc[j, name] = 0
#for statement check            
#df.head

In [9]:
work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

# table 4 odd columns

In [ ]:
#table 4
#(:,[1,3,5,7])/ co& ctrl_exp
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']
treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for o in outcome:
    for t in treatment:
        df_reg_t = df[(df[t]==1)&(df['ctrl_exp_' + t]==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
        idx = df_reg_t[df_reg_t[o].isnull()==True].index
        df_reg_o = df_reg_t.drop(idx)
        #vif cal
        #first, make a column list
        reg_col = []
        for i in work_list:
            reg_col.append(i)
        for j in year_list:
            reg_col.append(j)
        exog_var = ['fpsb_auction','reserve_price','municipality','fiscal_efficiency'] #id_auth빼봄
        exog = exog_var + reg_col 


        #check multicollinearity
        X = df_reg_o.loc[:,exog]
        vif = calc_vif(X)
        #print(vif)


        #delete from col list
        for i in range(len(vif)):
            if np.isnan(vif.loc[i, 'VIF']) == True:
                reg_col.remove(vif.loc[i, 'variables'])
            elif vif.loc[i,'VIF'] > 10:
                for j in exog_var:
                    if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                        exog_var.remove(vif.loc[i,'variables'])
                
        exog = exog_var + reg_col
        #exog.remove('id_auth')
        exog.remove(2000)
        exog.remove('OG01')
        exog.remove('municipality')

        #print(exog) #check
        print(mt.reg(df_reg_o, o, exog, fe_name = 'authority_code', cluster = 'auth_anno'))

# Table 4 even columns

In [ ]:
#table 4 discount
#(:,2)/ co/pr& ctrl_exp #trend_pa 들어가서 무지하게 큼

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df[t]==1)&(df['ctrl_exp_' + t]==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    for k in range(len(auth_list)):
        reg_col.append('trend_pa_'+str(k+1))
    exog_var = ['fpsb_auction','reserve_price','municipality','trend','trend_treat'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    print(mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno'))

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
